<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

**Мне нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаю такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосную корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.**

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
try:
    insurance = pd.read_csv('/Users/bogda/anaconda3/projects/praktikum/project8_insurance/insurance.csv')
except FileNotFoundError as e:
    print(e)
    insurance = pd.read_csv('/datasets/insurance.csv')

[Errno 2] No such file or directory: '/Users/bogda/anaconda3/projects/praktikum/project8_insurance/insurance.csv'


In [3]:
insurance

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


## Умножение матриц

* Сформулируем задачу.
* Изменится ли качество линейной регрессии при перемножении входящих данных на обратимую матрицу?

**Ответ:** Качество линейной регресии не изменится, докажем это утверждение.

**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

* Предсказания

$$a = Xw$$

* Формула обучения линейной регрессии

$$w = (X^TX)^{-1}X^Ty$$

* Пусть Y - умноженная на обратимую матрицу, матрица признаков X тогда новый  вектор весов линейной регрессии $w'$ равен

$$w' = (Y^TY)^{-1} Y^Ty$$

$$w' = ((XP)^T(XP))^{-1} (XP)^Ty$$

$$w' = P^{-1}((XP)^{T}X)^{-1} (XP)^Ty$$

$$w' = P^{-1}((XP)^{T}X)^{-1} P^TX^Ty$$

$$w' = P^{-1}(P^TX^TX)^{-1} P^TX^Ty$$

$$w' = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty$$

* Сократим 
$$(P^{T})^{-1} P^T = E , EX = X$$

$$w' = P^{-1}(X^TX)^{-1} X^Ty$$

Произведем замену
$$w = (X^TX)^{-1}X^Ty$$

$$w'= P^{-1}(X^TX)^{-1}X^Ty$$

$$w'= P^{-1}w$$

* Подставим $X$' и $w'$ в формулу для расчета предсказаний:

$$a'=X'w' = XPP^{-1}w = Xw = a$$

* чтд



## Алгоритм преобразования

**Алгоритм**

Генерируем обратимую матрицу признаков, умножаем исходную матрицу на сгенерированную. Используем данные в работе. Если нужно восстановить (хотя проще исходники не удалять:) умножаем на обратную матрицу.

**Обоснование**
см. доказательство выше


## Проверка алгоритма

In [37]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                            test_size=0.25, random_state=12345)
model = LinearRegression().fit(features_train, target_train)
predictions = model.predict(features_test)
r2_score(target_test, predictions)


0.43522757127025635

**Проверим сгенерированную матрицу на обратимость**

In [44]:
i = np.random.randint(1, 100, size=(4, 4))
while np.linalg.det(i) == 0:
    i = np.random.randint(1, 100, size=(4, 4))
print(i)    

[[77 99  5  9]
 [49 44 12  6]
 [37 63 39 88]
 [34 81 86 95]]


In [42]:
matrix = features.values @ i
matrix_train, matrix_test, target_mtrain, target_mtest = train_test_split(matrix, target, 
                                                                    test_size=0.25, random_state=12345)
model = LinearRegression().fit(matrix_train, target_mtrain)
predictions = model.predict(matrix_test)
r2_score(target_mtest, predictions)

0.43522757127035827

### Вывод
**Проверили на исходных данных что перемножение первичной матрицы признаков на обратимую не ведет к потери качества моделирования.**